# Single-Mode QCELS Utilities 

In [1]:
from __future__ import division
import sys,math,random,numpy as np,scipy,itertools,tqdm,warnings,matplotlib.animation,matplotlib.pylab as pl
from numpy import linalg as LA, inf
import numpy.typing as npt
import scipy.optimize as opt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

def fit_data(time_grid:npt.ArrayLike, data:npt.ArrayLike, bounds:npt.ArrayLike, n0:int=1, tol:float=1e-4):
    
    """
    Fit a sum of complex exponentials to the data through least squares.
    
    Parameters:
    - time_grid (numpy array): time grid
    - data (numpy array): Zn data
    - k (int): The model looks like sum_i=1^k overlap_i * exp(-1j*t*phase_i) where 
                 overlap_i, phase_i are the parameters to be optimized
    - n0 (int): number of starting guesses for optimizations
    - tol (float): tolerance on the cost function.''' 
    
    Returns:
    - x: optimal value of the estimated ground state energy
    - f_max: optimal value of the QCELS loss function
    """
    
    # r(theta) that minimizes L(r,theta) for fixed theta. The function is given in Eq 11 of the QCELS reference.
    def r_func(x):
        '''@args: x, phase parameter
           -out: r (1D array of complex numbers), amplitudes'''
        r = (1./len(time_grid)) * np.sum(data*np.exp(1j*x*time_grid))
        return r
    
    ######################################################
    # objective function (to be maximized)
    def f_func(x):
        return np.absolute( np.sum(data*np.exp(1j*x*time_grid)) )**2

    # negative objective function (to be minimized)
    def neg_f_func(x):
        return -f_func(x)
    ######################################################
    
    f_max = -np.Inf
    
    # initialization 
    xlo, xhi = bounds[0], bounds[1]
    x_bnd = opt.Bounds([xlo], [xhi])
    x0_array = np.linspace(start=xlo, stop=xhi, num=n0, endpoint=False)
    
    for i, x0 in enumerate(x0_array):
               
        # optimization
        s = opt.minimize(neg_f_func, x0, method='L-BFGS-B', bounds=x_bnd)
        
        # local maximum
        x_opt = s.x[0]
        f_opt = f_func(x_opt)

        # global maximum
        if f_opt > f_max:
            f_max = f_opt
            x_m = x_opt

    return x_m, f_max

def QCELS_gse(S_dat, time_grid, E, init_bounds):
    
    """
    Run QCELS to estimate the ground state energy

    Parameters:
    - S_dat: 1d array of complex-valued overlap matrix elements
    - time_grid[l] = l*dt: 1d array of timegrid
    - E: Energy spectrum of Hamiltonian (E_0, E_1, ..., E_{Hilbert space dimension})
    - init_bounds: initial guess of the energy parameter (theta_lo, theta_hi)
                   where theta_lo and theta_hi give lower and upper bounds on the energy estimate to be optimized
    
    Returns:
    - 1D array containing approximate ground state energy evaluated over the time grid
    """
    
    # tolerance on cost function
    tol = 1e-4                            
    gse = []
    for _ in range(len(time_grid)):
        
        n0 = 3*(_+1)                          
        theta, fit = fit_data(time_grid[:_+1], S_dat[:_+1], init_bounds, n0=n0, tol=tol)
        gse.append(theta)                         
    return np.array(gse) - E[0]

ModuleNotFoundError: No module named 'sklearn'